# Тема 1: Классификация состояния растений по изображениям

Данные: PlantVillage. Модель: EfficientNet (CNN). Метрики: Accuracy, F1, Confusion Matrix.

In [ ]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(ROOT))

from src.config import RAW_DIR, DATA_DIR
print("Data dir:", DATA_DIR)
print("Raw dir exists:", RAW_DIR.exists())

## 1. Скачивание данных (опционально)
Если данных ещё нет — загрузите датасет с Kaggle: [PlantVillage](https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset).  
Положите папку с классами (например `PlantVillage`) в `data/raw/`.

In [ ]:
# from src.data.download_data import download
# download()

## 2. Проверка датасета и загрузчиков

In [ ]:
from src.data.dataset import get_dataloaders, find_plantvillage_root

root = find_plantvillage_root()
print("Data root:", root)
train_loader, val_loader, class_names = get_dataloaders()
print("Number of classes:", len(class_names))
print("Sample classes:", class_names[:5])

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

batch = next(iter(train_loader))
imgs, labels = batch[0][:8], batch[1][:8]
grid = make_grid(imgs, nrow=4, normalize=True)
plt.figure(figsize=(10, 6))
plt.imshow(grid.permute(1, 2, 0))
plt.title("Labels: " + ", ".join(class_names[l] for l in labels.tolist()))
plt.axis("off")
plt.tight_layout()
plt.show()

## 3. Обучение модели
Запуск из корня проекта: `python -m src.train`

In [ ]:
from src.config import EPOCHS, LR, DEVICE, MODEL_DIR
import torch
from src.model import build_model
from src.data.dataset import get_dataloaders
from tqdm import tqdm

device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
train_loader, val_loader, class_names = get_dataloaders()
model = build_model(num_classes=len(class_names)).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(min(3, EPOCHS)):
    model.train()
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} done")

## 4. Оценка
После полного обучения: `python -m src.evaluate` — сохранит confusion matrix и метрики в `results/`.

In [ ]:
# Быстрая проверка метрик по текущей модели (если обучение было в ноутбуке):
from sklearn.metrics import classification_report, f1_score
import torch

model.eval()
preds, labels = [], []
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        preds.append(model(x).argmax(1).cpu())
        labels.append(y)
preds = torch.cat(preds).numpy()
labels = torch.cat(labels).numpy()
print("Accuracy:", (preds == labels).mean())
print("F1 macro:", f1_score(labels, preds, average="macro"))
print(classification_report(labels, preds, target_names=class_names))